# *ПРОЕКТ 4. АВИАРЕЙСЫ БЕЗ ПОТЕРЬ (ADVANCED)

ЗАДАЧА

Скачайте ваш датасет из Metabase. Используя либо Excel, либо Python, проанализируйте данные о доходности каждого рейса и затратах на них. Обратите внимание, что вам могут понадобиться дополнительные данные (например, стоимость топлива данной модели самолета на километр).

→ Шаг 1. Укажите номера рейсов, которые вы предлагаете убрать. Укажите предполагаемую прибыльность этих рейсов и среднюю прибыльность всех рейсов.

→ Шаг 2. Опишите, как вы проводили аналитику, какие данные были использованы для оценки доходности рейса и расходов на них.

### import


In [ ]:

import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

import os

import re
import datetime
from datetime import datetime, timedelta

In [ ]:
# проверка на работу в "google colab" или "локально"

if 'sample_data' in os.listdir():
    project_dir = r'/content/drive/My Drive/Colab Notebooks/module_4/'
    print('Обнаружена среда выполнения Google Colab.')
    print('project_dir =>', project_dir)
else:
    project_dir = ''
    print('НЕ обнаружена среда выполнения Google Colab. Выбран режим локальной работы.')

In [ ]:
RANDOM_SEED = 1314

### Функции обработки

### DATA


In [ ]:
# Выведем листинг проектного каталога
for dirname, _, filenames in os.walk(project_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
flight = pd.read_csv(project_dir + 'AAQ_flights.csv')
flight.sample(9)

In [ ]:
flight.columns

In [ ]:
len(flight.columns)

In [ ]:
# количество направлений
flight['city'].value_counts()

In [ ]:
# количество уникальных рейсов
flight['flight_no'].value_counts()

In [ ]:
flight['board_model'].value_counts()

In [ ]:
flight['flight_duration'].value_counts().sort_values(ascending=False)


[Цены на авиатопливо в 2017](https://favt.gov.ru/dejatelnost-ajeroporty-i-ajerodromy-ceny-na-aviagsm-statistika-aviagsm/?id=3241)

Анапа (будем считать, что все заправки проводились в этом городе)
*  декабрь: 47101 руб\тонна
*  январь: 41435 руб\тонна
*  февраль: 39553 руб\тонна



In [ ]:
# расход топлива за 1 час полета в кг
fuel_consumption = {'SU9': 1.7, '733': 2.4}

# стоимость топлива за 1000 кг
fuel_cost_2017 = {12: 47101, 1: 41435, 2: 39553}

# количество мест в самолете по классам обслуживания
cabin_size = {'SU9': {'Economy': 85, 'Business':12 },
              '733': {'Economy': 118, 'Business':12 }}

In [ ]:
flight['flight_hour'] = flight['flight_duration'].apply(lambda x: re.findall(r"\d+", str(x)))
flight['flight_hour'] = flight['flight_hour'].apply(lambda x: round(float(x[0]) + float(x[1])/60, 3))

In [ ]:
# Далее заполняем пропуски
flight.fillna(0, inplace=True)

In [ ]:
flight['flight_costs'] = flight.apply(lambda row: row['flight_hour'] * fuel_cost_2017[row['sched_month']] * fuel_consumption[row['aircraft_code']], axis=1 )

In [ ]:
flight['flight_profit'] = flight['total_amount'] - flight['flight_costs']

flight['flight_profit'] = flight['flight_profit'].apply(lambda x: int(x))

flight['total_amount'] = flight['total_amount'].apply(lambda x: int(x))

flight['flight_profit'] = flight['flight_profit'].apply(lambda x: int(x))

In [ ]:
flight.columns

In [ ]:
flight['flight_profit'].plot(kind='hist', grid=True, title='прдеположительная доходность')

In [ ]:
flight.groupby(by = ['sched_month', 'flight_no'])['flight_profit'].sum().plot(kind = 'bar', grid= True)

In [ ]:
# суммарная прибыль по рейсам с группировкой по месяцам
flight.groupby(by = ['sched_month', 'city'])['flight_profit'].sum().plot(kind = 'bar', grid= True)

In [ ]:
# средняя прибыль по городам с группировкой по месяцам
flight.groupby(by = ['sched_month', 'city'])['flight_profit'].mean().plot(kind = 'bar', grid= True)

In [ ]:
# число рейсов
flight.groupby(by = ['sched_month', 'city'])['flight_profit'].count().plot(kind = 'bar', grid= True)

In [ ]:
fig = plt.figure(figsize=(20, 6))
axes = fig.add_axes([0,0,1,1])
axes.plot(flight['scheduled_departure'], flight['total_amount']) 
axes.plot(flight['scheduled_departure'], flight['flight_costs'], linestyle='-') 
axes.plot(flight['scheduled_departure'], flight['flight_profit'], linestyle='--')


In [ ]:
# исппользуем из датасета только часть полей для анализа
fl = flight[['flight_id', 'sched_month', 'flight_no', 'flight_duration', 'city',
        'aircraft_code', 'total_amount', 'flight_hour', 'flight_costs', 'flight_profit' ]]

fl.sample(15, random_state=RANDOM_SEED)

In [ ]:
len(fl.columns), fl.columns

In [ ]:
fl.groupby(['city'])['total_amount'].sum()

In [ ]:
fl[fl['total_amount'] <= 0].groupby(['city'])['flight_no'].count()


In [ ]:
# выводим убыточный рейс
fl.query('total_amount <= 0')

In [ ]:
# суммарный предпологаемый убыток по рейсу в Новокузнецк
fl.query('flight_no == "PG0194"').groupby('flight_no', dropna='False')['flight_profit'].sum()

### Отрежем лишние данные


In [ ]:
fl = fl.query('total_amount > 0')
fl.shape

In [ ]:
# fl['flight_profit'].idxmin(axis="columns")

fl.iloc[fl['flight_profit'].idxmin()]

In [ ]:
fl['flight_profit'].min()

In [ ]:
fl['flight_profit'].max()

### Средняя предпологаемая прибыль по рейсам

In [ ]:
# средняя прибыль по рейсам и месяцам
fl.groupby(['flight_no', 'sched_month'])['flight_profit'].mean()

In [ ]:
# средняя прибыль по рейсам
fl.groupby(['flight_no'])['flight_profit'].mean()

In [ ]:
# суммарная прибыль по рейсам
fl.groupby(['flight_no'])['flight_profit'].sum()

In [ ]:
fl.query('flight_no == "PG0252"').sample(7)

In [ ]:
fl.query('flight_no == "PG0480"').sample(7)

In [ ]:
fl.groupby(by = ['flight_no'])['flight_profit'].sum().plot(kind = 'bar', grid= True)

In [ ]:
fl.groupby(by = ['flight_no'])['flight_profit'].mean().plot(kind = 'bar', grid= True)

In [ ]:
fl.groupby(by = ['aircraft_code'])['flight_profit'].sum().plot(kind = 'bar', grid= True)

In [ ]:
fl.groupby(by = ['aircraft_code'])['flight_profit'].mean().plot(kind = 'bar', grid= True)

In [ ]:
sns.jointplot(x = 'total_amount', y = 'flight_profit', data = fl, kind='reg')

In [ ]:
sns.jointplot(x = 'flight_hour', y = 'flight_profit', data = fl, kind='reg')

In [ ]:
sns.jointplot(x = 'flight_hour', y = 'flight_costs', data = fl, kind='reg')


## Выводы

Убыточный рейс (либо по нему нет данных о билетах): 
* PG0194 (Анапа-Новокузнецк). 
* Размер убытка: -4,481 млн руб

Средняя прибыль по рейсам (предположительно):
* PG0252(Moscow) = 1,48 млн руб
* PG0480(Belgorod) = 0,65 млн руб

Суммарная прибыль по рейсам (предположительно):
* PG0252(Moscow) = 87,360 млн руб
* PG0480(Belgorod) = 38,521 млн руб
